In [1]:
import pandas as pd
import numpy as np

In [2]:
auctions = pd.read_csv('auctions.csv', low_memory = False)

In [3]:
auctions["date"] =  pd.to_datetime(auctions["date"], errors = "coerce")

In [4]:
auctions_sorted = auctions.loc[:, ["device_id","date"]].sort_values(["device_id","date"], ascending = True)

In [5]:
auctions_sorted.head()

,device_id,date
36601349,40621409780134,2019-04-23 15:00:57.222979
30900602,41863526108385,2019-04-19 19:40:28.465866
14875535,41863526108385,2019-04-20 02:52:26.892880
32062481,41863526108385,2019-04-20 02:59:02.509230
45613542,41863526108385,2019-04-20 03:06:01.675788


In [6]:
auctions_sorted['timediff'] = np.where(auctions_sorted['device_id'] == auctions_sorted['device_id'].shift(), auctions_sorted['date'] - auctions_sorted['date'].shift(1), np.nan)

In [7]:
auctions_sorted['time_to_reappear'] = auctions_sorted['timediff'].dt.total_seconds()

In [10]:
def calculate_time_to_reappear(df, feature_date_name, feature_id_name):
    df_sorted = df.sort_values([feature_id_name, feature_date_name], ascending = True)
    df_sorted['time_to_reappear'] = np.\
    where(df_sorted[feature_id_name] == \
          df_sorted[feature_id_name].shift(), df_sorted[feature_date_name] - df_sorted[feature_date_name].shift(1), np.nan)
    df_sorted['time_to_reappear'] = df_sorted['time_to_reappear'].dt.total_seconds()
    return df_sorted

In [9]:
auctions_sorted.head()

,device_id,date,timediff,time_to_reappear
36601349,40621409780134,2019-04-23 15:00:57.222979,NaT,NaN
30900602,41863526108385,2019-04-19 19:40:28.465866,NaT,NaN
14875535,41863526108385,2019-04-20 02:52:26.892880,07:11:58.427014,25918.427014
32062481,41863526108385,2019-04-20 02:59:02.509230,00:06:35.616350,395.616350
45613542,41863526108385,2019-04-20 03:06:01.675788,00:06:59.166558,419.166558
